In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu,True)

In [ ]:
gpus

In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
from sklearn import metrics

In [ ]:
from numpy.random import seed

seed(1)
from tensorflow import random

random.set_seed(2)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Futurense Hackaton/AAPL_sentiment.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)

df.dropna(inplace=True)
df.tail()

In [ ]:
df.index

In [ ]:
df = df[["Adj Close", "ts_polarity", "twitter_volume"]]
df.head()

In [ ]:
df.shape

In [ ]:
df["Pct_change"] = df["Adj Close"].pct_change()

df.dropna(inplace = True)
df.head()

In [ ]:
def window_data(df, window, feature_col_number1, feature_col_number2, feature_col_number3, target_col_number):
    X_close = []
    X_polarity = []
    X_volume = []
    y = []
    for i in range(len(df) - window):
        close = df.iloc[i:(i + window), feature_col_number1]
        ts_polarity = df.iloc[i:(i + window), feature_col_number2]
        tw_vol = df.iloc[i:(i + window), feature_col_number3]
        target = df.iloc[(i + window), target_col_number]
        
        X_close.append(close)
        X_polarity.append(ts_polarity)
        X_volume.append(tw_vol)
        y.append(target)
        
    return np.hstack((X_close,X_polarity,X_volume)), np.array(y).reshape(-1, 1)

In [ ]:
window_size = 3

feature_col_number1 = 0
feature_col_number2 = 1
feature_col_number3 = 2
target_col_number = 0
X, y = window_data(df, window_size, feature_col_number1, feature_col_number2, feature_col_number3, target_col_number)

In [ ]:
X_split = int(0.7 * len(X))
y_split = int(0.7 * len(y))



X_train = X[: X_split]
X_test = X[X_split:]
y_train = y[: y_split]
y_test = y[y_split:]

In [ ]:
X_split

In [ ]:
df.iloc[641]

**Scaling Data**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
x_train_scaler = MinMaxScaler()
x_test_scaler = MinMaxScaler()
y_train_scaler = MinMaxScaler()
y_test_scaler = MinMaxScaler()

x_train_scaler.fit(X_train)
y_train_scaler.fit(y_train)

X_train = x_train_scaler.transform(X_train)
y_train = y_train_scaler.transform(y_train)

x_test_scaler.fit(X_test)
y_test_scaler.fit(y_test)

X_test = x_test_scaler.transform(X_test)
y_test = y_test_scaler.transform(y_test)

**Reshaping Data**

In [ ]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
model = Sequential()

number_units = 9
dropout_fraction = 0.2

model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
    
model.add(Dropout(dropout_fraction))


model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))

model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))

model.add(Dense(1))

In [ ]:
model.compile(optimizer="adam", loss="mean_squared_error")

In [ ]:
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=30, shuffle=False, verbose=1)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
predicted = model.predict(X_test)

In [ ]:
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predicted)))
print('R-squared :', metrics.r2_score(y_test, predicted))



In [ ]:
predicted_prices = y_test_scaler.inverse_transform(predicted)
real_prices = y_test_scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices): ]) 

stocks.head()

In [ ]:
stocks.tail()

In [ ]:
print(stocks.columns)

In [ ]:
stock_monthly = stocks.loc['2019-06-27':'2019-08-30']

In [ ]:
stock_monthly.to_csv('/content/drive/MyDrive/Futurense Hackaton/real-pred_2month_sentiment.csv', index=True)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(30,10))
plt.plot(stock_monthly.index, stock_monthly["Real"], label="Real")
plt.plot(stock_monthly.index, stock_monthly["Predicted"], label="Predicted")

plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()

plt.show()

In [ ]:
import os
model.save(os.path.join('/content/drive/MyDrive/models','Apple_stock_market_model'))